- parsed_chart에서 모든 song_id를 읽는다.
- 하나씩 가져와서 html로 저장한다.

In [2]:
import os
import requests
import time
import json
from tqdm import tqdm
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager

ModuleNotFoundError: No module named 'selenium'

## get song ids

In [2]:
path = "parsed_chart_html"
json_files = os.listdir(path)

In [26]:
song_ids = set()
for json_file in json_files:
    with open(os.path.join(path, json_file), "r") as f:
        for line in tqdm(f):
            doc = json.loads(line)
            song_id = doc["title_num"]
            song_ids.add(song_id)
    
print("num song ids: ", len(song_ids))

5200it [00:00, 136529.06it/s]
1900it [00:00, 136432.82it/s]
5200it [00:00, 150824.16it/s]
5141it [00:00, 209564.38it/s]
5200it [00:00, 187834.31it/s]
5300it [00:00, 163413.64it/s]
5182it [00:00, 163340.35it/s]
5200it [00:00, 172948.65it/s]
5200it [00:00, 175902.93it/s]
5300it [00:00, 159861.14it/s]
5200it [00:00, 162929.40it/s]
5200it [00:00, 193436.75it/s]
5200it [00:00, 184638.14it/s]
5184it [00:00, 179352.58it/s]
5198it [00:00, 187632.79it/s]
5200it [00:00, 186466.10it/s]
5299it [00:00, 134830.64it/s]
5192it [00:00, 122684.27it/s]
5193it [00:00, 129890.93it/s]

num song ids:  12146


In [27]:
song_ids = sorted(song_ids)

## crawl song info

In [28]:
def initBrowser():
    #
    chrome_options = webdriver.ChromeOptions()
    chrome_options.add_argument('--incognito') # incognito 시크릿 모드입니다.
    browser = webdriver.Chrome(ChromeDriverManager().install(), options=chrome_options)
    #
    return browser
    

def quitBrowser(browser):
    #
    browser.quit()
    #
    return None

def getHtmlwithBrowser(browser, url):
    #
    browser.get(url)
    html = browser.page_source
    time.sleep(1)
    #
    return html

In [29]:
song_html_path = "htmls/songs"
file_names = os.listdir(song_html_path)
alread_crawled_song_ids = set([_.replace(".html", "") for _ in file_names])

# 이미 parse된 songs를 읽는다.
try: 
    with open("song_infos.jsonl", "r") as f:
        lines = f.readlines()
        docs = [json.loads(_) for _ in lines]
        parsed_song_ids = [_["song_id"] for _ in docs]
except:
    parsed_song_ids = []

In [30]:
print("song_id nums: {}".format(len(song_ids)))
print("already crawled: {}".format(len(alread_crawled_song_ids)))
print("already parsed: {}".format(len(parsed_song_ids)))

song_ids = [_ for _ in song_ids if _ not in alread_crawled_song_ids and _ not in parsed_song_ids]
print("song_ids to crawl: {}".format(len(song_ids)))

song_id nums: 12146
already crawled: 5124
already parsed: 14681
song_ids to crawl: 0


In [25]:
#browser.quit()
browser = initBrowser()


for song_id in tqdm(song_ids):
    
    url = "https://www.melon.com/song/detail.htm?songId={}".format(song_id)
    html = getHtmlwithBrowser(browser, url)

    
    if "잘못된 경로로 접근" in html:
        print("잘못된 경로: {}".format(song_id))
        browser.quit()
        browser = initBrowser()
        time.sleep(1)
        continue
        
    if song_id not in html:
        print("song_id가 없음.")
        continue
        
    #
    with open(os.path.join(song_html_path, "{}.html".format(song_id)), "w") as of:
        print(html, file=of)
        
    # 2초이상 쉬어야 끊기지 않는다.
    time.sleep(1)

/var/folders/03/p2hj2jdx6wq13p2tcscyfbwh0000gn/T/ipykernel_2301/4026465351.py:5: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  browser = webdriver.Chrome(ChromeDriverManager().install(), options=chrome_options)
100%|████████████████████████████████████████████████████████████| 210/210 [13:08<00:00,  3.76s/it]
